[View in Colaboratory](https://colab.research.google.com/github/tcglarry/geo/blob/master/geo_Unet_colab.ipynb)

In [7]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

gpg: keybox '/tmp/tmp4c6iw7qb/pubring.gpg' created
gpg: /tmp/tmp4c6iw7qb/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [10]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [11]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

aoi_aoi_test1.ipynb  Sample upload.txt			      Untitled1.ipynb
aoi_test	     TAHU PowerPoint Template.pdf	      Untitled2.ipynb
Capstone_PixNet.zip  TAHU PowerPoint Template - 柱狀圖 1.ods  Untitled3.ipynb
Colab Notebooks      test for excel			      Untitled4.ipynb
PiXNet		     Untitled0.ipynb			      Untitled5.ipynb


In [12]:

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Conv2DTranspose
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import regularizers

Using TensorFlow backend.


In [0]:
src = 'drive/aoi_test/t_brain/geo/'

In [14]:
data= np.load(src+'train_data.npy')
label = np.load(src+'train_labels.npy')
print (np.mean(label))
label = label[:,:,:,np.newaxis]
print (data.shape)
print (label.shape)

5.66184075511905
(4200, 21, 21, 20)
(4200, 20, 20, 1)


In [0]:

data_mean = np.mean(data)
data_std = np.std(data)
label_max = np.max(label)
label_min = np.min(label)

train_data = (data - data_mean)/data_std
#label_data = (label-label_min)/(label_max - label_min)
label_data = np.log(label)

-3.586322865788844

In [0]:
train_x,val_x,train_y,val_y = train_test_split(train_data,label_data,test_size=0.2, shuffle=True)

In [0]:
def conv_block(ch, activation= 'relu', padding='same',kernel_regularizer=regularizers.l2(0.01)):       
    return Conv2D(ch,(3,3),activation= activation, padding =padding )


In [0]:
#def build_model_1(IMAGE_HEIGHT=IMAGE_HEIGHT,IMAGE_WIDTH=IMAGE_WIDTH,ch=ch):
def build_model_1(ch=8):
    #inputs = Input((IMAGE_HEIGHT,IMAGE_WIDTH,ch))
    inputs = Input((21,21,20))

    conv0 = Conv2D(32,(2,2),padding='valid')(inputs)
    print ('conv0',conv0.get_shape())
    conv1 =  conv_block(ch)(conv0)
    conv1 = conv_block(ch)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    print ('pool1',pool1.get_shape())
    
    conv2 =  conv_block(ch*2)(pool1)
    conv2 = conv_block(ch*2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    print ('poo12',pool2.get_shape())
    
    conv_test = Conv2D(64,(2,2),padding='valid')(pool2)
    print ('conv_test',conv_test.get_shape())
    
    
    
    conv3 =  conv_block(ch*4)(conv_test)
    conv3 = conv_block(ch*4)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    print ('poo13',pool3.get_shape())
    
    conv4 =  conv_block(ch*8)(pool3)
    conv4 = conv_block(ch*8)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    print ('poo14',pool4.get_shape())
    
    conv5 =  conv_block(ch*16)(pool4)
    conv5 = conv_block(ch*16)(conv5)

    
    up6 = concatenate([UpSampling2D(size=(2,2))(conv5), conv4], axis=3)
    conv6 = conv_block(ch*8)(up6)
    conv6 = conv_block(ch*8)(conv6)
    
    up7 = concatenate ([UpSampling2D(size=(2,2))(conv6), conv3],  axis=3)
    conv7 = conv_block(ch*4)(up7)
    conv7 = conv_block(ch*4)(conv7)
    print ('conv7',conv7.get_shape())
    conv7 = Conv2DTranspose(ch*4,(2,2),padding='valid')(conv7)
    print ('conv7',conv7.get_shape())
    
    up8 = concatenate([UpSampling2D(size=(2,2))(conv7), conv2], axis=3)
    conv8 = conv_block(ch*2)(up8)
    conv8 = conv_block(ch*2)(conv8)
    
    up9 = concatenate([UpSampling2D(size=(2,2))(conv8), conv1],  axis=3)
    conv9 = conv_block(ch)(up9)
    conv9 = conv_block(ch)(conv9)
    
    
    
    conv10 = Conv2D(1,(1,1))(conv9)
    
  
    model = Model(inputs= inputs, outputs=conv10)
    
    return model
    
    

In [40]:
model = build_model_1()
model.summary()

conv0 (?, 20, 20, 32)
pool1 (?, 10, 10, 8)
poo12 (?, 5, 5, 16)
conv_test (?, 4, 4, 64)
poo13 (?, 2, 2, 32)
poo14 (?, 1, 1, 64)
conv7 (?, 4, 4, 32)
conv7 (?, ?, ?, 32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 21, 21, 20)   0                                            
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 20, 20, 32)   2592        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 20, 20, 8)    2312        conv2d_22[0][0]                  
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 20

In [0]:



def gram_matrix(x):
  #features = K.batch_flatten(K.permute_dimensions(x[0], (2, 0, 1)))
  x = K.batch_flatten(x)
  gram = K.dot(x, K.transpose(x))
  print(gram)
  #return K.expand_dims(gram, axis=0)
  return gram
  


def my_loss(y_true,y_pred):
  diff_1  = K.mean(K.square(y_pred - y_true), axis=-1)
  diff_2 = K.mean(K.square(gram_matrix(y_pred) - gram_matrix(y_true)), axis=-1)
  
  total_loss = diff_1 * 1.0 + 0.05 * diff_2
  return total_loss
  



In [0]:
model.compile (optimizer='Adam', loss = 'mse', metrics =['mae'])

In [0]:

# train the model on the new data for a few epochs

use_model= False
if use_model:
    

    if os.path.isfile(src+'water_first_try_2.h5'):

      model = load_model(src+'water_first_try_2.h5')
      print ('model laoded')
    else:
      print ('model not exist')


In [44]:

checkpoint = ModelCheckpoint(filepath=src+'water_first_try_2_prepro.h5', monitor = 'val_loss', save_best_only=True, mode= 'auto')
earlystop = EarlyStopping(patience=10)
callback_list = [checkpoint, earlystop]
model.fit(x=train_x, y=train_y, batch_size=32, epochs=100, callbacks = callback_list, verbose=1, validation_split=0.2,  shuffle=True)


Train on 2688 samples, validate on 672 samples
Epoch 1/100
2688/2688 [==============================] - 6s 2ms/step - loss: 1.0227 - mean_absolute_error: 0.7992 - val_loss: 0.7317 - val_mean_absolute_error: 0.6820
Epoch 2/100
2688/2688 [==============================] - 3s 1ms/step - loss: 0.6125 - mean_absolute_error: 0.6206 - val_loss: 0.4907 - val_mean_absolute_error: 0.5526
Epoch 3/100
2688/2688 [==============================] - 3s 1ms/step - loss: 0.4429 - mean_absolute_error: 0.5238 - val_loss: 0.3869 - val_mean_absolute_error: 0.4886
Epoch 4/100
2688/2688 [==============================] - 3s 1ms/step - loss: 0.3450 - mean_absolute_error: 0.4593 - val_loss: 0.3075 - val_mean_absolute_error: 0.4360
Epoch 5/100
2688/2688 [==============================] - 3s 1ms/step - loss: 0.2846 - mean_absolute_error: 0.4157 - val_loss: 0.2573 - val_mean_absolute_error: 0.3962
Epoch 6/100
2688/2688 [==============================] - 3s 1ms/step - loss: 0.2482 - mean_absolute_error: 0.3870 - va

In [45]:
error = model.evaluate(val_x,val_y)
print (error)

840/840 [==============================] - 0s 312us/step
[0.07144045120193845, 0.2014346203633717]


In [48]:

print ('label mean = ', np.mean(label_data))
print ('per_element_error = ', np.sqrt(error[0]/840.))
print ('error percentage = ', str(round(np.sqrt(error[0]/400.)/np.mean(label_data)*100.,2))+'%')
     

label mean =  1.2508584707231873
per_element_error =  0.009222155723733139
error percentage =  1.07%


In [0]:
!cp drive/aoi_test/t_brain/geo/test2.npy .

In [65]:
print (os.path.isfile('test2.npy'))
test = np.load('test2.npy')
test_data = (test - data_mean)/data_std
test_data.shape

True


(1800, 21, 21, 20)

In [66]:
prediction = model.predict(test_data)
prediction = np.exp(prediction)
pred = prediction.reshape(1800,-1)
print (pred.shape)


pred_df = pd.DataFrame(pred)
pred_df.keys()


(1800, 400)


RangeIndex(start=0, stop=400, step=1)

In [0]:
test_df = pd.read_csv(src+'upload_sample.csv')
col_names = test_df.keys().tolist()

In [68]:
final = pd.concat([test_df['ID'],pred_df],axis=1)
final.columns = col_names
final.to_csv(src+'test_submit.csv', index=False)
print ('submit file saved')

submit file saved


In [69]:
test_see_df = pd.read_csv(src+'test_submit.csv')
test_see_df

,ID,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,...,loc391,loc392,loc393,loc394,loc395,loc396,loc397,loc398,loc399,loc400
0,F0000009,3.637450,3.397361,1.306265,0.975866,0.771085,0.737831,0.817702,1.063377,1.649419,...,13.923022,6.668416,3.553622,4.393536,4.321436,3.027205,4.289143,4.127999,5.373748,4.044850
1,F0000011,5.257187,4.587291,3.364237,1.732740,1.410341,2.149034,1.633356,1.296906,0.848735,...,3.427324,2.060781,1.165417,1.534841,1.598410,0.656144,0.788388,0.506130,0.376934,0.600194
2,F0000012,4.919220,5.033795,3.522160,1.924298,0.563182,0.473420,0.788400,0.520226,0.525983,...,2.487790,3.353358,3.049527,2.195502,1.443538,0.671322,0.958591,1.592936,1.776364,1.466742
3,F0000014,1.974877,2.616695,2.078576,0.750662,0.485556,1.104774,2.902478,2.877553,1.395398,...,2.654470,2.506235,9.764881,11.702678,15.215517,6.685521,5.648466,4.998720,4.276497,5.740833
4,F0000017,0.399199,0.732806,1.364653,1.809032,0.948285,0.994955,2.472865,4.325450,4.097332,...,5.043988,8.814609,12.740374,11.531866,10.934320,2.903601,1.486856,0.501627,2.232121,1.452096
5,F0000018,2.546994,2.649576,1.937539,1.369597,1.025648,0.590963,0.571080,0.564285,1.085754,...,2.029347,2.833265,1.136839,1.339821,3.788051,5.917206,2.854777,1.764061,3.617316,4.944537
6,F0000024,1.310133,1.436392,1.892699,1.903182,1.464933,1.076052,1.056197,1.490342,1.962202,...,0.935146,1.457265,3.041853,1.919180,2.134631,2.659752,1.693090,1.717085,2.763929,3.042111
7,F0000036,3.434881,3.412232,2.955921,5.507609,6.812491,7.020388,8.145371,8.155762,8.424997,...,10.010551,4.567230,4.933782,6.330770,8.497945,10.430654,12.616276,5.326504,4.950913,4.957148
8,F0000046,8.925598,14.172565,22.632565,20.162525,8.765533,3.759120,1.763196,2.195614,3.421595,...,4.935758,4.768136,6.096569,5.413257,4.770568,10.121556,9.358714,9.386264,7.934508,11.944634
9,F0000047,1.666242,2.025659,2.608924,3.002703,2.506790,1.863883,1.982960,1.843743,1.993239,...,6.984561,8.164933,4.994627,3.366159,6.321540,7.785236,6.100740,8.110456,13.695886,10.892977


In [0]:
!cp drive/aoi_test/t_brain/geo/test_submit.csv .

In [0]:
from google.colab import files
files.download('test_submit.csv')